# Training SmolLM2-135M

This notebook demonstrates how to train the custom `SmolLM2` model (imported from `model.py`) on custom dataset. It includes:
1. Training for 5000 steps.
2. Generating text every 500 steps.
3. Saving a checkpoint.
4. Resuming training from the checkpoint.

In [ ]:
# Install PyTorch with CUDA support for Windows (assuming CUDA 12.1)
# If this fails or you have a different CUDA version, check https://pytorch.org/get-started/locally/
!pip uninstall -y torch torchvision
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets

> **IMPORTANT**: After running the cell above, you **MUST** restart the Jupyter Kernel for the changes to take effect. Go to **Kernel > Restart Kernel** in the menu.

In [6]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoConfig
from datasets import load_dataset
from model import SmolLM2  # Import our custom model
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cpu":
    print("WARNING: You are running on CPU. Training will be very slow. Please ensure you have a GPU and the correct PyTorch version installed.")

PyTorch Version: 2.5.1+cu121
CUDA Available: True
CUDA Device: NVIDIA RTX 5000 Ada Generation Laptop GPU
CUDA Version: 12.1
Using device: cuda


## 1. Load Model and Tokenizer

In [7]:
# Load Custom Tokenizer
tokenizer_path = "./custom_tokenizer"

# Check if custom tokenizer exists
if os.path.exists(tokenizer_path):
    print(f"Loading custom tokenizer from {tokenizer_path}")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    print(f"Custom tokenizer not found at {tokenizer_path}. Please run train_tokenizer.ipynb first.")
    # Fallback to default for safety, or raise error. 
    # Assuming user wants custom, we should probably raise error or warn heavily.
    # For this flow, let's fallback but warn.
    print("Falling back to default tokenizer (HuggingFaceTB/SmolLM2-135M)...")
    model_id = "HuggingFaceTB/SmolLM2-135M"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

# Ensure pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Padding token set to: {tokenizer.pad_token}")

model_id = "HuggingFaceTB/SmolLM2-135M"
config = AutoConfig.from_pretrained(model_id)

# Update config vocab size to match tokenizer
config.vocab_size = len(tokenizer)
print(f"Model vocab size updated to: {config.vocab_size}")

# Initialize model from scratch
model = SmolLM2(config).to(device)
print("Model initialized.")

Loading custom tokenizer from ./custom_tokenizer
Model vocab size updated to: 5001
Model initialized.


## 2. Prepare Dataset (Chunked)
We concatenate text and split into chunks to allow the model to learn context across lines.

In [8]:
# Load dataset by reading the full file to preserve newlines and formatting
with open("input-1.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

from datasets import Dataset
dataset = Dataset.from_dict({"text": [full_text]})

# Double check tokenizer padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

block_size = 256 # Context window size

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
        
    # Split by chunks of max_len
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

def tokenize_function(examples):
    return tokenizer(examples["text"])

# Tokenize all text
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Group into chunks
lm_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    # num_proc=4, # Disabled to avoid Windows multiprocessing 'spawn' issues with global variables
)

lm_dataset = lm_dataset.with_format("torch")

# Create dataloader
train_dataloader = DataLoader(lm_dataset, batch_size=4, shuffle=True)
print(f"Dataset prepared. Number of chunks: {len(lm_dataset)}")

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset prepared. Number of chunks: 1308


## 3. Training Loop

In [9]:
# Optimization: Enable TF32 for faster matrix multiplications on Ampere+ GPUs
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Optimization: Use torch.compile (PyTorch 2.0+)
# Windows support for torch.compile can be tricky, so we wrap it in a try-except or check os
if os.name != 'nt': # torch.compile often has issues on Windows currently, skipping for safety or try 'inductor'
     print("Compiling model with torch.compile...")
     model = torch.compile(model)
else:
    print("Skipping torch.compile on Windows to avoid potential compatibility issues.")

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, fused=True if torch.cuda.is_available() else False)
loss_fn = torch.nn.CrossEntropyLoss()

# Optimization: Mixed Precision Training
scaler = torch.cuda.amp.GradScaler()

def generate_text(model, tokenizer, prompt="The meaning of life is", max_new_tokens=50, temperature=0.7, top_k=50):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs.input_ids
    
    for _ in range(max_new_tokens):
        with torch.no_grad():
            # Auto-cast is not strictly necessary for inference but can speed it up
            with torch.cuda.amp.autocast():
                logits = model(input_ids)
            next_token_logits = logits[:, -1, :] / temperature
            
            # Top-k sampling
            top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k, dim=-1)
            probs = torch.nn.functional.softmax(top_k_logits, dim=-1)
            next_token_index = torch.multinomial(probs, num_samples=1)
            next_token = top_k_indices.gather(-1, next_token_index)
            
            input_ids = torch.cat([input_ids, next_token], dim=1)
            
            if next_token.item() == tokenizer.eos_token_id:
                break
            
    print(f"Generated: {tokenizer.decode(input_ids[0], skip_special_tokens=True)}")
    model.train()

steps = 0
max_steps = 5000
save_path = "checkpoint_5000.pt"

model.train()
print("Starting training...")

# Loop indefinitely until max_steps is reached
while steps < max_steps:
    for batch in train_dataloader:
        if steps >= max_steps:
            break
            
        input_ids = batch["input_ids"].to(device)
        # Shift labels for causal LM
        labels = input_ids.clone()
        
        optimizer.zero_grad()
        
        # Optimization: Mixed Precision Context
        with torch.cuda.amp.autocast():
            logits = model(input_ids)
            
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            
            loss = loss_fn(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))
        
        # Optimization: Scaled Backward Pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        steps += 1
        
        if steps % 100 == 0:
            print(f"Step {steps}: Loss {loss.item()}")
            
        if steps % 500 == 0:
            print(f"\n--- Step {steps} Generation ---")
            generate_text(model, tokenizer)
            print("-----------------------------\n")

# Save Checkpoint
torch.save(model.state_dict(), save_path)
print(f"Checkpoint saved to {save_path}")

Skipping torch.compile on Windows to avoid potential compatibility issues.
Starting training...


C:\Users\sidhe\AppData\Local\Temp\ipykernel_23008\2461595262.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\sidhe\AppData\Local\Temp\ipykernel_23008\2461595262.py:65: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Step 100: Loss 5.271072864532471
Step 200: Loss 5.146054267883301
Step 300: Loss 4.636077404022217
Step 400: Loss 4.05342960357666
Step 500: Loss 4.514355182647705

--- Step 500 Generation ---


C:\Users\sidhe\AppData\Local\Temp\ipykernel_23008\2461595262.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Generated: The meaning of life is
The people; and therefore the very royal soldiers,
The noble duke, I'll make you both the people.

PETRUCHIO:
Well, I must:
What you have some mil'dly?

GLOUCESTER:
What
-----------------------------

Step 600: Loss 3.852552652359009
Step 700: Loss 3.67338228225708
Step 800: Loss 3.858555316925049
Step 900: Loss 3.88484263420105
Step 1000: Loss 3.3133487701416016

--- Step 1000 Generation ---
Generated: The meaning of life is
drumbling; and in the middle of love
of I am the end of the hand: I have sworn
without you have done.

LEONTES:
I pray you, sir, I'll swear
To say you
-----------------------------

Step 1100: Loss 3.7228200435638428
Step 1200: Loss 3.4301235675811768
Step 1300: Loss 3.7488114833831787
Step 1400: Loss 3.0603678226470947
Step 1500: Loss 3.1305861473083496

--- Step 1500 Generation ---
Generated: The meaning of life is it not;
And she, for all the fairest of this earth;
And, by the other of heaven she's love,
Persuitute it is not be

## 4. Resume Training
Now we simulate stopping and resuming by loading the checkpoint and training for 50 more steps.

In [11]:
print("Resuming training...")
model = SmolLM2(config).to(device)
model.load_state_dict(torch.load(save_path))
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

extra_steps = 50
current_step = 0

# Re-create dataloader for resume (in real scenario you'd want to skip already seen data)
# Note: In this simple example, we restart the dataloader from the beginning.
# In a real resume scenario, you'd want to save the dataloader state or skip 'steps' batches.
train_dataloader_resume = DataLoader(lm_dataset, batch_size=4, shuffle=True)

for batch in train_dataloader_resume:
    if current_step >= extra_steps:
        break
        
    input_ids = batch["input_ids"].to(device)
    labels = input_ids.clone()
    
    optimizer.zero_grad()
    logits = model(input_ids)
    
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    loss = loss_fn(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))
    loss.backward()
    optimizer.step()
    
    current_step += 1
    if current_step % 10 == 0:
        print(f"Resume Step {current_step}: Loss {loss.item()}")

print(f"\n--- Step {steps + extra_steps} Generation ---")
generate_text(model, tokenizer, prompt="First Citizen", max_new_tokens=1024)
print("-----------------------------\n")

torch.save(model.state_dict(), f"checkpoint_{steps + extra_steps}.pt")
print(f"Checkpoint saved to checkpoint_{steps + extra_steps}.pt")

print("Resumed training completed.")

Resuming training...


C:\Users\sidhe\AppData\Local\Temp\ipykernel_23008\3952903180.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


Resume Step 10: Loss 0.14603516459465027
Resume Step 20: Loss 0.12318285554647446
Resume Step 30: Loss 0.08086450397968292
Resume Step 40: Loss 0.11517078429460526
Resume Step 50: Loss 0.13376004993915558

--- Step 5050 Generation ---


C:\Users\sidhe\AppData\Local\Temp\ipykernel_23008\2461595262.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Generated: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't: let it be done: away, away!

Second Citizen:
One word.

First Citizen:
We are coniled and we will.

MENENIUS:
Ay, we'll show 'em good friends,' we'll show themselves:
'Tis very strangely.

Second Citizen:
And 'twas a purposed, we'll keep no tears;
But we shall have vengeance and meet the belly;
Where, they shall stand, will we do good men, we'll
With our senators: we'll promption 'O' good,'
Their very wayward.

MENENIUS:
What we will, to do
Our Rome will in justice w: the gods do o' the city
In every mattling aparted, is as an art,
It cannot miss't: the other lady,
In peace what it has, in